In [1]:
import pandas as pd
import win32com.client as win32
import time
import requests
import json
import openpyxl
import pandas_datareader.data as web
from datetime import date, timedelta
from openpyxl import load_workbook

Puxar Valor Dolar

In [2]:
requisicao = requests.get(r'https://economia.awesomeapi.com.br/all/USD-BRL')
cotacao = requisicao.json()
cotacao_dolar = float(cotacao['USD']['bid'])
print(cotacao_dolar)

5.1144


Puxar quantidade Dolar existente

Atualizar planilha com bolsa Down Jones

In [3]:
carteira_df = pd.read_excel(
    r'C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\Carteira_usa.xlsx')
today = date.today()
yesterday = today - timedelta(days = 1)
cotacoes_carteira = pd.DataFrame()

try:
    for ativo in carteira_df['Ativos']:
        cotacoes_carteira[ativo] = web.DataReader('{}'.format(ativo), data_source='yahoo', start=yesterday, end=yesterday)['Adj Close']
except:
        for ativo in carteira_df['Ativos']:
            cotacoes_carteira[ativo] = web.DataReader('{}'.format(ativo), data_source='yahoo', start=today, end=today)['Adj Close']

valor_investido = pd.DataFrame()
for ativo in carteira_df['Ativos']:
    valor_investido[ativo] = cotacoes_carteira[ativo] * carteira_df.loc[carteira_df['Ativos'] == ativo, 'Qtde'].values[0]

valor_investido['TOTAL'] = valor_investido[list(valor_investido.columns)].sum(axis=1)

valor_investido.to_excel(r'C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\Carteira_usa_dia.xlsx')

In [4]:

valor_investido = pd.DataFrame()
for ativo in carteira_df['Ativos']:
    valor_investido[ativo] = cotacoes_carteira[ativo] * carteira_df.loc[carteira_df['Ativos'] == ativo, 'Qtde'].values[0]

valor_investido['TOTAL'] = valor_investido[list(valor_investido.columns)].sum(axis=1)

valor_investido.to_excel(r'C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\Carteira_usa_dia.xlsx')

In [5]:

valor_investido = pd.DataFrame()
for ativo in carteira_df['Ativos']:
    valor_investido[ativo] = cotacoes_carteira[ativo] * carteira_df.loc[carteira_df['Ativos'] == ativo, 'Qtde'].values[0]

valor_investido['TOTAL'] = valor_investido[list(valor_investido.columns)].sum(axis=1)

valor_investido.to_excel(r'C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\Carteira_usa_dia.xlsx')

In [6]:
dolar_wb = load_workbook(r"C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\Carteira_usa_dia.xlsx", data_only=True)
sh = dolar_wb["Sheet1"]
dolartt= (sh["k2"].value)
print(dolartt)

16413.3965950531


Puxando montante em criptos

In [7]:
cripto_wb = load_workbook(r"C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\Carteira_criptos_dia.xlsx", data_only=True)
sh = cripto_wb["Sheet1"]
criptott= (sh["F2"].value)
print(criptott)

4591.626357654291


Abrindo Data Frames

Abrir Info.xls e alterar o valor do dolar cotação e dolar montante total

In [8]:

wb = load_workbook(filename=r'C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\infos.xlsx')
ws = wb.worksheets[1]
ws['I4'] = cotacao_dolar
ws['I2'] = dolartt
ws['H2'] = criptott
wb.save(r'C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\infos.xlsx')


Excel file, open, refresh and close

In [9]:
# Start an instance of Excel
xlapp = win32.DispatchEx("Excel.Application")

# Open the workbook in said instance of Excel
wb = xlapp.workbooks.open(r'C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\infos.xlsx')

# Optional, e.g. if you want to debug
xlapp.Visible = True

# Refresh all data connections.
wb.RefreshAll()
wb.Save()

# Quit
xlapp.Quit()

In [14]:
base_dash_df = pd.read_excel(
    r'base_dash.xlsx', index_col=[0])
infos_df = pd.read_excel(
    r'C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\infos.xlsx', index_col=[0],sheet_name=0)
display(base_dash_df)
display(infos_df)


,Data,Fundos de Investimentos,Fundos Imobiliários,Ações,COE,Renda Fixa,Disponível,Cripto,Dow Jones,Total
NaN,20/07/2022,117484.14,487055.63,14198.81,5000,13415.65,13175.88,4407.690000,68900.000000,723637.800000
NaN,27/07/2022,122779.91,533358.48,13490.60,5000,13416.94,1096.02,4407.690000,68900.000000,762449.640000
NaN,28/07/2022,122899.81,533217.58,14025.40,5000,13414.85,1096.02,4407.690000,68900.000000,762961.350000
NaN,08/08/2022,123428.76,535534.83,14904.06,5000,13400.25,2591.90,4407.690000,68900.000000,765575.590000
NaN,09/08/2022,123489.84,538180.19,14763.26,5000,13398.16,2591.90,4407.690000,68900.000000,768139.140000
NaN,11/08/2022,123615.01,535089.68,14398.16,5000,13392.04,2591.90,4842.390000,90577.650000,786480.230000
NaN,13/08/2022,123678.97,540386.52,14431.89,5000,13389.83,5706.41,4842.390000,90577.650000,792307.250000
NaN,16/08/2022,123742.97,542476.77,14873.15,5000,13385.41,5706.41,4842.390000,90577.650000,794898.340000
NaN,17/08/2022,123809.72,543304.23,14836.99,5000,13383.20,6958.82,4965.270000,92513.250000,797812.660000
NaN,18/08/2022,123939.31,545387.46,14860.73,5000,13382.43,6958.82,4823.180000,94364.016500,808715.946500


,Data,Fundos de Investimentos,Fundos Imobiliários,Ações,COE,Renda Fixa,Disponível,Cripto,Dow Jones,Total
NaN,31/10/2022,134390.28,545032.14,11959.95,5000,13413.3,0,4591.626358,84043.155925,798430.452283


Concatenar com novas infos

In [15]:
bases_df = [base_dash_df, infos_df]
base_dash_df = pd.concat(bases_df, )
print(base_dash_df)
base_dash_df.to_excel(r'base_dash.xlsx')


           Data  Fundos de Investimentos  Fundos Imobiliários     Ações   COE  \
NaN  20/07/2022                117484.14            487055.63  14198.81  5000   
NaN  27/07/2022                122779.91            533358.48  13490.60  5000   
NaN  28/07/2022                122899.81            533217.58  14025.40  5000   
NaN  08/08/2022                123428.76            535534.83  14904.06  5000   
NaN  09/08/2022                123489.84            538180.19  14763.26  5000   
NaN  11/08/2022                123615.01            535089.68  14398.16  5000   
NaN  13/08/2022                123678.97            540386.52  14431.89  5000   
NaN  16/08/2022                123742.97            542476.77  14873.15  5000   
NaN  17/08/2022                123809.72            543304.23  14836.99  5000   
NaN  18/08/2022                123939.31            545387.46  14860.73  5000   
NaN  19/08/2022                124004.04            544898.73  14289.14  5000   
NaN  22/08/2022             

Plotar Gráfico

#base_dash_df.plot(x="Data", figsize=(20, 10))
#display(base_dash_df)

Procurando data do Posição Detalhada

In [12]:
posicao_detalhada_df = pd.read_excel(
    r'C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\PosicaoDetalhada.xlsx', index_col=[0])
posicao = posicao_detalhada_df.columns[4]
data_posicao_detalhada = posicao[17:27]
print(data_posicao_detalhada)


31/10/2022


Enviando por email

'''outlook = win32.Dispatch('outlook.application')

data = data_posicao_detalhada
email = 'beligolli@hotmail.com'


mail = outlook.CreateItem(0)
mail.To = email
mail.Subject = 'Dash_Financeiro - {}.'.format(data)
mail.Body = """
Segue Dash Financeiro até o dia {} 

att.,
""".format(data)
#ou mail.HTMLBody = '<p>Corpo do Email em HTML</p>'

# Anexos (pode colocar quantos quiser):
attachment = r'C:\Users\belig\OneDrive\Python\MeuProjeto\Projetos\dash_investimentos\Resumo_Investimentos\base_dash.xlsx'
mail.Attachments.Add(attachment)

mail.Send()'''

Codigo para informar que a operação foi concluida

In [13]:
# from tkinter import *
# from tkinter import messagebox
# janela = Tk()
# messagebox.showinfo('Status', 'Processo Concluído.')
# janela.destroy